# Likelihood and uncertainty
A notebook to illustrate the relationships between the likelihood function and different uncertainty estimates.

## Authors:
**David W. Hogg** (NYU)

## License:
- Copyright 2025 the author. All code is licensed for re-use under the open-source *MIT License*.

## Notes:
- Some overlap with `basic_inference_example.ipynb`.

## To-do:
- Make all plots consistent across all noteboooks, so they are publication-ready.

## Bugs:
- Various things hard-coded.

In [ ]:
import numpy as np
import pylab as plt
from matplotlib import rcParams
import scipy.optimize as op

In [ ]:
rcParams['figure.figsize'] = [4.0, 4.0]

In [ ]:
# set default global stuff (apologies)

N = 7
p = 2
prior_bounds = np.array([[1., 1.5 * np.pi], [0., 2. * np.pi]])
assert prior_bounds.shape == (p, 2)
true_omega = 2.13 # hard-coded global magic variable

In [ ]:
# make fake data

def expectation(ts, pars):
    amp, phi = pars
    return amp * np.cos(true_omega * ts - phi)

def make_fake_data(seed=17):
    rng = np.random.default_rng(seed)
    ts = np.sort(7. * rng.uniform(size=N))
    ivars = 0.25 * (1. + 1. * rng.uniform(size=N)) # magic
    truepars = np.array([2.59, 2.0344]) # magic
    return ts, expectation(ts, truepars) + rng.normal(size=N) / np.sqrt(ivars), ivars, truepars

In [ ]:
ts, ys, ivars, true_pars = make_fake_data()
print(ts.shape, ys.shape, true_pars)

In [ ]:
def plot(ts, ys, ivars, true_pars, ml_pars, samples, title):
    plt.errorbar(ts, ys, yerr=1./np.sqrt(ivars), fmt="ko")
    plot_ts = np.linspace(0., 7., 1000)
    if samples is not None:
        for sample in samples:
            plt.plot(plot_ts, expectation(plot_ts, sample), "r-", lw=1, alpha=0.45)
    if true_pars is not None:
        plt.plot(plot_ts, expectation(plot_ts, true_pars), "b-", lw=1, alpha=0.45)
    if ml_pars is not None:
        plt.plot(plot_ts, expectation(plot_ts, ml_pars), "r-", lw=2, alpha=0.9)
    plt.xlabel("time")
    plt.ylabel("data value")
    plt.title(title)

plot(ts, ys, ivars, true_pars, None, None, "data and true expectation")

In [ ]:
# define likelihood in terms of phase

def negative_log_likelihood(pars, ts, ys, ivars):
    return 0.5 * np.sum(ivars * (ys - expectation(ts, pars)) ** 2)

In [ ]:
res = op.minimize(negative_log_likelihood, true_pars, args=(ts, ys, ivars))
print(res)
ml_pars = np.zeros(4) + np.nan
ml_pars_covar = np.zeros((4,4)) + np.nan
if res.success:
    ml_pars = res.x
    ml_pars_covar = res.hess_inv
ml_pars[1] = np.arctan2(np.sin(ml_pars[1]), np.cos(ml_pars[1]))
print(ml_pars)

In [ ]:
plot(ts, ys, ivars, true_pars, ml_pars, None, "maximum-likelihood estimate")

In [ ]:
# define functions in terms of amplitudes

def design_matrix(ts):
    return np.vstack([np.cos(true_omega * ts), np.sin(true_omega * ts)]).T

def ml_amplitudes(ts, ys, ivars):
    X = design_matrix(ts)
    return np.linalg.solve(X.T @ (ivars[:, None] * X), X.T @ (ivars * ys))

def amps_to_pars(amps):
    a, b = amps
    return np.array([np.sqrt(a ** 2 + b ** 2), np.arctan2(b, a)])

In [ ]:
# check that everyone is cool

ml_amps = ml_amplitudes(ts, ys, ivars)
schml_pars = amps_to_pars(ml_amps)
print(schml_pars, np.allclose(ml_pars, schml_pars))

In [ ]:
# make a likelihood image
# bug: loop

damp = 0.03
dphi = 0.03
amplim = (0., 8.)
ampvec = np.arange(amplim[0] + 0.5 * damp, amplim[1], damp)
philim = (0., 2. * np.pi)
phivec = np.arange(philim[0] + 0.5 * dphi, philim[1], dphi)
print(ampvec.shape, phivec.shape)
amps, phis = np.meshgrid(ampvec, phivec)
lls = np.zeros_like(amps) + np.nan
for i in range(lls.shape[0]):
    for j in range(lls.shape[1]):
        lls[i, j] = - negative_log_likelihood((amps[i, j], phis[i, j]), ts, ys, ivars)
print(np.min(lls), np.max(lls))

In [ ]:
mlls = np.max(lls)
plt.imshow(np.exp(lls - mlls), interpolation="nearest", origin="lower",
           extent=amplim+philim,
           vmin=0, vmax=1, cmap="gray_r", aspect="auto")
plt.contour(ampvec, phivec, lls - mlls, origin="lower",
            levels=[-1.,], colors="r", linestyles="solid", linewidths=0.5, alpha=0.9)
plt.scatter([ml_pars[0], ], [ml_pars[1], ], marker="x", c="r",
            s=20., alpha=0.9)
plt.scatter([true_pars[0], ], [true_pars[1], ], marker="x", c="b",
            s=20., linewidths=0.5, alpha=0.5)
plt.xlabel("amplitude $A$")
plt.ylabel("phase $\phi$")

In [ ]:
ayvec = np.arange(-amplim[1] + 0.5 * damp, amplim[1], damp)
ays, bees = np.meshgrid(ayvec, ayvec)
lls2 = np.zeros_like(ays) + np.nan
for i in range(lls2.shape[0]):
    for j in range(lls2.shape[1]):
        lls2[i, j] = - negative_log_likelihood(amps_to_pars((ays[i, j], bees[i, j])), ts, ys, ivars)
print(np.min(lls2), np.max(lls2))

In [ ]:
mlls2 = np.max(lls2)
plt.imshow(np.exp(lls2 - mlls2), interpolation="nearest", origin="lower",
           extent=[-amplim[1], amplim[1], -amplim[1], amplim[1]],
           vmin=0, vmax=1, cmap="gray_r", aspect="auto")
plt.contour(ayvec, ayvec, lls2 - mlls2, origin="lower",
            levels=[-1.,], colors="r", linestyles="solid", linewidths=0.5, alpha=0.9)
plt.scatter([ml_amps[0], ], [ml_amps[1], ], marker="x", c="r",
            s=20., alpha=0.9)
plt.scatter([true_pars[0] * np.cos(true_pars[1]), ], [true_pars[0] * np.sin(true_pars[1]), ], marker="x", c="b",
            s=20., linewidths=0.5, alpha=0.5)
plt.xlabel("cosine amplitude $a$")
plt.ylabel("sine amplitude $b$")